In [1]:
import pandas as pd
import fnmatch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
import os.path
from pathlib import Path
import glob
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
import imageio
import IPython.display as ipd
import matplotlib.image as mpimg
import pydub
from scipy.io.wavfile import read, write
import librosa
import librosa.display
import IPython
from IPython.display import Audio
import scipy
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape,\
Conv2DTranspose, LeakyReLU, Conv1D, AveragePooling1D, MaxPooling1D, MaxPool1D, GlobalAvgPool1D, GRU
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

In [2]:
data_path = "/kaggle/input/heartbeat-sound/Heartbeat_Sound"
print(os.listdir(data_path))

['artifact', 'unlabel', 'extrastole', 'extrahls', 'normal', 'murmur']


In [3]:
tarin_data      = data_path 
unlabel_data    = data_path  + "/unlabel/"

normal_data     = tarin_data + '/normal/'
murmur_data     = tarin_data + '/murmur/'
extrastole_data = tarin_data + '/extrastole/'
artifact_data   = tarin_data + '/artifact/'
extrahls_data   = tarin_data + "/extrahls/"

In [4]:
def add_noise(data,x):
    noise = np.random.randn(len(data))
    data_noise = data + x * noise
    return data_noise

def shift(data,x):
    return np.roll(data, x)

def stretch(data, rate):
    data = librosa.effects.time_stretch(data, rate=rate)
    return data

def pitch_shift (data , rate):
    data = librosa.effects.pitch_shift(data, sr=220250, n_steps=rate)
    return data

In [5]:
def load_file_data (folder, file_names, duration=10, sr=22050):
    '''
        Extract MFCC feature from the Sound data from the audio data. 
        Augmentation of sound data by adding Noise, streaching and shifting.
        52 features are extracted from each audio data and used to train the model.
        
        Args: 
            dir_: Input directory to the Sound input file.
        
        Returns:
            data: list of features extracted from the sound file.
    '''
    input_length=sr*duration
    features = 52
    data = []
    for file_name in file_names:
        try:
            sound_file = folder+file_name
            X, sr = librosa.load( sound_file, sr=sr, duration=duration) 
            dur = librosa.get_duration(y=X, sr=sr)
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                X = librosa.util.fix_length(X, size=input_length)  
                
            # extract normalized mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=features).T,axis=0) 
            feature = np.array(mfccs).reshape([-1,1])
            data.append(feature)
            
            stretch_data_1 = stretch (X, 0.8)
            mfccs_stretch_1 = np.mean(librosa.feature.mfcc(y=stretch_data_1, sr=sr, n_mfcc=features).T,axis=0) 
            feature_1 = np.array(mfccs_stretch_1).reshape([-1,1])
            data.append(feature_1)
            
            stretch_data_2 = stretch (X, 1.2) 
            mfccs_stretch_2 = np.mean(librosa.feature.mfcc(y=stretch_data_2, sr=sr, n_mfcc=features).T,axis=0) 
            feature_2 = np.array(mfccs_stretch_2).reshape([-1,1])
            data.append(feature_2)
            
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)  
            print(e)
        
    return data

In [6]:
CLASSES = ['normal','abnormal']
NB_CLASSES=len(CLASSES)

# Map integer value to text labels
label_to_int = {k:v for v,k in enumerate(CLASSES)}
print (label_to_int)
print (" ")
int_to_label = {v:k for k,v in label_to_int.items()}
print(int_to_label)

{'normal': 0, 'abnormal': 1}
 
{0: 'normal', 1: 'abnormal'}


In [7]:
SAMPLE_RATE = 22050
# 10 seconds
MAX_SOUND_CLIP_DURATION=10

artifact_files = fnmatch.filter(os.listdir(artifact_data), 'artifact*.wav')
artifact_sounds = load_file_data (folder=artifact_data, file_names = artifact_files, duration=MAX_SOUND_CLIP_DURATION)
artifact_labels = [1 for items in artifact_sounds]

normal_files = fnmatch.filter(os.listdir(normal_data), 'normal*.wav')
normal_sounds = load_file_data(folder=normal_data,file_names=normal_files, duration=MAX_SOUND_CLIP_DURATION)
normal_labels = [0 for items in normal_sounds]

extrahls_files = fnmatch.filter(os.listdir(extrahls_data), 'extrahls*.wav')
extrahls_sounds = load_file_data(folder=extrahls_data,file_names=extrahls_files, duration=MAX_SOUND_CLIP_DURATION)
extrahls_labels = [1 for items in extrahls_sounds]

murmur_files = fnmatch.filter(os.listdir(murmur_data), 'murmur*.wav')
murmur_sounds = load_file_data(folder=murmur_data,file_names=murmur_files, duration=MAX_SOUND_CLIP_DURATION)
murmur_labels = [1 for items in murmur_sounds]


extrastole_files = fnmatch.filter(os.listdir(extrastole_data), 'extrastole*.wav')
extrastole_sounds = load_file_data(folder=extrastole_data,file_names=extrastole_files, duration=MAX_SOUND_CLIP_DURATION)
extrastole_labels = [1 for items in extrastole_sounds]

print ("Loading Done")

fixing audio lenght : artifact__201106131834.wav
fixing audio lenght : artifact__201106101955.wav
fixing audio lenght : artifact__201106220340.wav
fixing audio lenght : artifact__201105190800.wav
fixing audio lenght : artifact__201106040722.wav
fixing audio lenght : artifact__201106070949.wav
fixing audio lenght : artifact__201106221254.wav
fixing audio lenght : artifact__201106010602.wav
fixing audio lenght : artifact__201106021541.wav
fixing audio lenght : artifact__201106010559.wav
fixing audio lenght : artifact__201105061143.wav
fixing audio lenght : artifact__201106110909.wav
fixing audio lenght : artifact__201106161219.wav
fixing audio lenght : artifact__201105040918.wav
fixing audio lenght : artifact__201106211041.wav
fixing audio lenght : artifact__201106211430.wav
fixing audio lenght : artifact__201105280851.wav
fixing audio lenght : artifact__201105051017.wav
fixing audio lenght : artifact__201106131835.wav
fixing audio lenght : artifact__201106050353.wav
fixing audio lenght 

In [8]:
Bunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Bunlabelledtest*.wav')
Bunlabelledtest_sounds = load_file_data(folder=unlabel_data,file_names=Bunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
Bunlabelledtest_labels = [-1 for items in Bunlabelledtest_sounds]

Aunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Aunlabelledtest*.wav')
Aunlabelledtest_sounds = load_file_data(folder=unlabel_data,file_names=Aunlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
Aunlabelledtest_labels = [-1 for items in Aunlabelledtest_sounds]


print ("Loading of unlabel data done")

fixing audio lenght : Bunlabelledtest__231_1308748318393_C.wav
fixing audio lenght : Bunlabelledtest__152_1306779561195_C.wav
fixing audio lenght : Bunlabelledtest__235_1308749032454_C1.wav
fixing audio lenght : Bunlabelledtest__107_1305654946865_B.wav
fixing audio lenght : Bunlabelledtest__148_1306768801551_D.wav
fixing audio lenght : Bunlabelledtest__171_1307971016233_F.wav
fixing audio lenght : Bunlabelledtest__169_1307970398039_C1.wav
fixing audio lenght : Bunlabelledtest__148_1306768801551_A.wav
fixing audio lenght : Bunlabelledtest__179_1307990076841_D1.wav
fixing audio lenght : Bunlabelledtest__159_1307018640315_C.wav
fixing audio lenght : Bunlabelledtest__300_1311772096120_D.wav
fixing audio lenght : Bunlabelledtest__151_1306779785624_A1.wav
fixing audio lenght : Bunlabelledtest__181_1308052613891_C.wav
fixing audio lenght : Bunlabelledtest__174_1307987737137_C.wav
fixing audio lenght : Bunlabelledtest__286_1311170606028_B.wav
fixing audio lenght : Bunlabelledtest__175_13079879

In [9]:
x_data = np.concatenate((artifact_sounds, normal_sounds,extrahls_sounds,murmur_sounds,extrastole_sounds))

y_data = np.concatenate((artifact_labels, normal_labels,extrahls_labels,murmur_labels,extrastole_labels))

test_x = np.concatenate((Aunlabelledtest_sounds,Bunlabelledtest_sounds))
test_y = np.concatenate((Aunlabelledtest_labels,Bunlabelledtest_labels))

print ("combined training data record: ",len(y_data), len(test_y))

combined training data record:  1755 741


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=42, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=42, shuffle=True)

# One-Hot encoding for classes
y_train = np.array(tf.keras.utils.to_categorical(y_train, len(CLASSES)))
y_test = np.array(tf.keras.utils.to_categorical(y_test, len(CLASSES)))
y_val = np.array(tf.keras.utils.to_categorical(y_val, len(CLASSES)))
test_y=np.array(tf.keras.utils.to_categorical(test_y, len(CLASSES)))

In [11]:
crnn_model = Sequential()

crnn_model.add(Conv1D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(52, 1)))
crnn_model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
crnn_model.add(BatchNormalization())
crnn_model.add(Dropout(0.2))

crnn_model.add(Conv1D(32, kernel_size=3, strides=1, padding='same', activation='relu'))
crnn_model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
crnn_model.add(BatchNormalization())
crnn_model.add(Dropout(0.2))

crnn_model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
crnn_model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
crnn_model.add(BatchNormalization())
crnn_model.add(Dropout(0.2))

crnn_model.add(GRU(64, return_sequences=True))
crnn_model.add(GRU(32))

crnn_model.add(Dense(64, activation='relu'))

crnn_model.add(Dense(2, activation='softmax'))

crnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 52, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 26, 32)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 32)           128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 26, 32)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 26, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 32)           0

In [12]:
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0001)
crnn_model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

cb = [EarlyStopping(patience=150,monitor='val_accuracy',mode='max',restore_best_weights=True),
      ModelCheckpoint("/kaggle/working/Heart_LSTM_CNN_1.h5",save_best_only=True)]
crnn = crnn_model.fit(x_train, y_train, 
                         validation_data=(x_val, y_val),
                         batch_size=16, epochs=10000, 
                         callbacks = cb )

Epoch 1/10000
79/79 [==============================] - 10s 33ms/step - loss: 0.6714 - accuracy: 0.6105 - val_loss: 0.6903 - val_accuracy: 0.5390
Epoch 2/10000
79/79 [==============================] - 1s 17ms/step - loss: 0.6698 - accuracy: 0.6105 - val_loss: 0.7000 - val_accuracy: 0.5390
Epoch 3/10000
79/79 [==============================] - 1s 17ms/step - loss: 0.6625 - accuracy: 0.6081 - val_loss: 0.7017 - val_accuracy: 0.5390
Epoch 4/10000
79/79 [==============================] - 1s 18ms/step - loss: 0.6681 - accuracy: 0.6112 - val_loss: 0.7031 - val_accuracy: 0.5390
Epoch 5/10000
79/79 [==============================] - 1s 18ms/step - loss: 0.6668 - accuracy: 0.6144 - val_loss: 0.6923 - val_accuracy: 0.5390
Epoch 6/10000
79/79 [==============================] - 1s 18ms/step - loss: 0.6605 - accuracy: 0.6120 - val_loss: 0.6908 - val_accuracy: 0.5390
Epoch 7/10000
79/79 [==============================] - 2s 19ms/step - loss: 0.6540 - accuracy: 0.6215 - val_loss: 0.6746 - val_accuracy

In [13]:
crnn_model.evaluate(x_test, y_test)

11/11 [==============================] - 0s 6ms/step - loss: 0.0661 - accuracy: 0.9829


[0.06606197357177734, 0.9829059839248657]